# Решение задачи "Сумма на отрезке"

In [149]:
import random
import os
import math

- N - число элементов массива: [1, 100000]
- K - число запросов: [0, 100000]
- i - номер элемента в массиве: [1, 100000]
- x - значение: [0, 1000000000]
- A i x - присвоить значение
- Q l r - найти сумму

## Вспомогательные функции

In [174]:
def generate_files(N_min, N_max, K_max, x_max, null_cases=True):
    N = random.randint(N_min, N_max)
    if null_cases:
        K = random.randint(0, K_max)
    else:
        K = random.randint(N, K_max)
    with open('sum.in', 'w') as fi, open('sum_right.out', 'w') as fo:
        fi.write(f'{N} {K}\n')
        if K:
            array = [random.randint(0, x_max) for i in range(N+1)]
            array[0] = 0
            #print('generated array:', array)
            for j in range(1, N+1):
                num = array[j]
                if num:
                    fi.write(f'A {j} {num}\n')
                    K -= 1
            for j in range(K):
                l = random.randint(1, N)
                r = random.randint(l, N)
                s = sum(array[l:r+1])
                K -= 1
                if K:
                    fi.write(f'Q {l} {r}\n')
                    fo.write(str(s)+'\n')
                else:
                    fi.write(f'Q {l} {r}')
                    fo.write(str(s))


def compare_files():                   
    with open('sum_right.out', 'r') as fr, open('sum.out', 'r') as fo:
        if not (fr.read() == fo.read()):
            print('False')
            return

## Реализация

In [150]:
def get_data():
    with open('sum.in', 'r') as fi:
        for line in fi:
            yield line
            

def get_double_array(array, N):
    # вычисляем ближайшее число с основанием 2
    x = 2**math.ceil(math.log(N, 2))
    new_array = [0 for i in range(2*x)]
    for i in range(1, N+1):
        new_array[i+x-1] = array[i]
    return new_array, 2*x


def fill_array(array, N):
    for i in range (N-2, 0, -2):
        array[i//2] = array[i] + array[i+1]

        
def get_sum(array, ind_l, ind_r):
    result = 0
    while ind_l <= ind_r:
        # если элемент с индексом ind_l - правый потомок (нечетный индекс), добавляем к результату
        if ind_l % 2 == 1:
            result += array[ind_l]
        # если элемент с индексом ind_r - левый потомок (четный индекс), добавляем к результату
        if ind_r % 2 == 0:
            result += array[ind_r]
        ind_l = (ind_l + 1) // 2
        ind_r = (ind_r - 1) // 2
    return result


def get_result():
    lines = get_data()
    tmp = next(lines).split()
    N, K = int(tmp[0]), int(tmp[1])
    if not K:
        open('sum.out', 'w').close()
        return
    array = [0 for i in range(N+1)]
    tmp = next(lines).split()
    while tmp[0] == 'A':
        array[int(tmp[1])] = int(tmp[2])
        K -= 1
        if K:
            tmp = next(lines).split()
        else:
            open('sum.out', 'w').close()
            return
    array, N = get_double_array(array, N)
    fill_array(array, N)
    with open('sum.out', 'w') as fo:
        while True:
            s = get_sum(array, int(tmp[1])+N//2-1, int(tmp[2])+N//2-1)
            K -= 1
            if K:
                fo.write(str(s)+'\n')
                tmp = next(lines).split()
            else:
                fo.write(str(s))
                return

## Тестирование

In [172]:
N_min = 1
N_max = 10**5
K_max = 10**5
x_max = 10**9

In [155]:
%%timeit -r 20 -n 1 -o -q

generate_files(N_min, N_max, K_max, x_max, null_cases=False)
get_result()
compare_files()

os.remove('sum.in')
os.remove('sum_right.out')
os.remove('sum.out')

<TimeitResult : 1.17 s ± 890 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)>